In [379]:

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression


In [380]:
# turn into function
def read_file(file_path):
    return pd.read_csv(file_path)
card_df = read_file("Credit_card.csv")
label_df = pd.read_csv('Credit_card_label.csv')
merged_encoded = pd.merge(df, label_df, on='Ind_ID')
df.head()

,Ind_ID,GENDER,Car_Owner,Propert_Owner,CHILDREN,Annual_income,Type_Income,EDUCATION,Marital_status,Housing_type,Birthday_count,Employed_days,Mobile_phone,Work_Phone,Phone,EMAIL_ID,Type_Occupation,Family_Members
0,5008827,M,Y,Y,0,180000.0,Pensioner,Higher education,Married,House / apartment,-18772.0,365243,1,0,0,0,NaN,2
1,5009744,F,Y,N,0,315000.0,Commercial associate,Higher education,Married,House / apartment,-13557.0,-586,1,1,1,0,NaN,2
2,5009746,F,Y,N,0,315000.0,Commercial associate,Higher education,Married,House / apartment,-15661.5,-586,1,1,1,0,NaN,2
3,5009749,F,Y,N,0,166500.0,Commercial associate,Higher education,Married,House / apartment,-13557.0,-586,1,1,1,0,NaN,2
4,5009752,F,Y,N,0,315000.0,Commercial associate,Higher education,Married,House / apartment,-13557.0,-586,1,1,1,0,NaN,2


In [383]:
#1
def clean_data(merged_encoded: pd.DataFrame, 
                          categorical_cols: List[str] = ['Housing_type', 'Marital_status', 'Propert_Owner', 'Type_Income', 
                                                         'Type_Occupation', 'Car_Owner', 'GENDER', 'EDUCATION'],
                          numerical_cols: List[str] = ['Annual_income', 'Birthday_count'], 
                          occupation_col: str = 'Type_Occupation') -> pd.DataFrame:
    for col in numerical_cols:
        df[col] = df[col].fillna(df[col].median())
    merged_encoded.dropna(subset=[occupation_col], inplace=True)
    merged_encoded['GENDER'] = merged_encoded['GENDER'].fillna(merged_encoded['GENDER'].mode()[0])
    data_encoded = pd.get_dummies(merged_encoded, columns=categorical_cols, drop_first=True)
    merged_encoded.dropna(subset=[occupation_col], inplace=True)

    data_encoded = data_encoded.replace([np.inf, -np.inf], np.nan).fillna(0)
    data_encoded= data_encoded.astype(int)
    return data_encoded
data_encoded

,Ind_ID,GENDER,Car_Owner,Propert_Owner,CHILDREN,Annual_income,Type_Income,EDUCATION,Marital_status,Housing_type,Birthday_count,Employed_days,Mobile_phone,Work_Phone,Phone,EMAIL_ID,Type_Occupation,Family_Members,label
0,5008827,M,Y,Y,0,180000.0,Pensioner,Higher education,Married,House / apartment,-18772.0,365243,1,0,0,0,NaN,2,1
1,5009744,F,Y,N,0,315000.0,Commercial associate,Higher education,Married,House / apartment,-13557.0,-586,1,1,1,0,NaN,2,1
2,5009746,F,Y,N,0,315000.0,Commercial associate,Higher education,Married,House / apartment,NaN,-586,1,1,1,0,NaN,2,1
3,5009749,F,Y,N,0,NaN,Commercial associate,Higher education,Married,House / apartment,-13557.0,-586,1,1,1,0,NaN,2,1
4,5009752,F,Y,N,0,315000.0,Commercial associate,Higher education,Married,House / apartment,-13557.0,-586,1,1,1,0,NaN,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1543,5028645,F,N,Y,0,NaN,Commercial associate,Higher education,Married,House / apartment,-11957.0,-2182,1,0,0,0,Managers,2,0
1544,5023655,F,N,N,0,225000.0,Commercial associate,Incomplete higher,Single / not married,House / apartment,-10229.0,-1209,1,0,0,0,Accountants,1,0
1545,5115992,M,Y,Y,2,180000.0,Working,Higher education,Married,House / apartment,-13174.0,-2477,1,0,0,0,Managers,4,0
1546,5118219,M,Y,N,0,270000.0,Working,Secondary / secondary special,Civil marriage,House / apartment,-15292.0,-645,1,1,1,0,Drivers,2,0


In [384]:
#2
def credit_card_approval(data_encoded: pd.DataFrame, 
                               feature_cols: List[str], 
                               target_col: str) -> float:
    X = data_encoded[feature_cols]
    y = data_encoded[target_col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    l_g = LinearRegression()
    l_g.fit(X_train, y_train)
    y_pred = l_g.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return mse, r2
print(f'MSE: {mse}, R2: {r2}')

MSE: 0.11819873066694385, R2: -0.010801559404991723


this method dit not good for prediction because the MSE is so low.

In [394]:
#3
def optimal_k(data_encoded: pd.DataFrame, 
                   target_col: str, 
                   k_range: Tuple[int, int] = (1, 21), 
                   test_size: float = 0.2, 
                   random_state: int = 42) -> float:
    X = data_encoded.drop(columns=[target_col])
    y = data_encoded[target_col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,
                                                        random_state=42)
    for k in k_range:
        knn.fit(X_train_sc, y_train)
        score = knn.score(X_test_sc, y_test)
        if score > best_score:
            optimal_k = k
            best_score = score
    knn.fit(X_train_, y_train)
    return best_k, test_accuracy, 
print(f"Optimal k: {best_k}")
print(f"Test Accuracy: {test_accuracy}")

Optimal k: 1
Test Accuracy: 0.8490566037735849


this model preformed pretty well it was very accurate 

In [386]:
#4
def logistic_regression(data_encoded: pd.DataFrame) -> float:
    X = data_encoded.drop(columns=['label', 'Ind_ID'])
    y = data_encoded['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                        random_state=42)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.fit_transform(X_test)
    regression = LogisticRegression(random_state=42).fit(X_train, y_train)
    y_pred = regression.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    #report = classification_report(y_test, y_pred)
    return accuracy
print(accuracy)


<function accuracy at 0x1444ad080>


logistic regression did not work well with the other data sets becasue it acuraccy is very low.

In [387]:
#5.2
def credit_card_approval(data_encoded: pd.DataFrame, 
                               feature_cols: List[str], 
                               target_col: str) -> float:
    merged_encoded = pd.merge(df, label_df, on='Ind_ID')
    X = data_encoded[feature_cols]
    y = data_encoded[target_col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    l_g = LinearRegression()
    l_g.fit(X_train, y_train)
    y_pred = l_g.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return mse, r2
print(mse, r2)

0.11819873066694385 -0.010801559404991723


In [388]:
#2 didn't really do anthing the performance is the same.

In [393]:
#5.3
def optimal_k(data_encoded: pd.DataFrame, 
                   target_col: str, 
                   k_range: Tuple[int, int] = (1, 21), 
                   test_size: float = 0.2, 
                   random_state: int = 42) -> float:
    X = data_encoded.drop(columns=[target_col])
    y = data_encoded[target_col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,
                                                        random_state=42)
    for k in k_range:
        knn.fit(X_train_sc, y_train)
        score = knn.score(X_test_sc, y_test)
        if score > best_score:
            optimal_k = k
            best_score = score
    knn.fit(X_train_, y_train)
    return best_k, test_accuracy, 
print(f"Optimal k: {best_k}")
print(f"Test Accuracy: {test_accuracy}")

Optimal k: 1
Test Accuracy: 0.8490566037735849


I got the same result as last time beacuse the data set is alredy orginized.


In [390]:
#5.4
def logistic_regression(data_encoded: pd.DataFrame) -> float:
    X = data_encoded.drop(columns=['label', 'Ind_ID'])
    y = data_encoded['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                        random_state=42)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.fit_transform(X_test)
    regression = LogisticRegression(random_state=42).fit(X_train, y_train)
    y_pred = regression.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy
print(accuracy)


<function accuracy at 0x1444ad080>


Normalizing the function didnt do anything to change the anwser.